In [25]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
import os

In [26]:
df = pd.read_csv("../data/processed/final_merged.csv")
df.head()

,anilist_id,mal_id,title_romaji,title_english,title_native,description,episodes,genres,popularity,averageScore,coverImage,combined,rating_count,avg_rating
0,16498,16498.0,Shingeki no Kyojin,Attack on Titan,進撃の巨人,"Several hundred years ago, humans were nearly ...",25.0,"['Action', 'Drama', 'Fantasy', 'Mystery']",938026,85.0,https://s4.anilist.co/file/anilistcdn/media/an...,"Attack on Titan ['Action', 'Drama', 'Fantasy',...",938026,8.5
1,101922,38000.0,Kimetsu no Yaiba,Demon Slayer: Kimetsu no Yaiba,鬼滅の刃,"It is the Taisho Period in Japan. Tanjiro, a k...",26.0,"['Action', 'Adventure', 'Drama', 'Fantasy', 'S...",890351,82.0,https://s4.anilist.co/file/anilistcdn/media/an...,"Demon Slayer: Kimetsu no Yaiba ['Action', 'Adv...",890351,8.2
2,1535,1535.0,DEATH NOTE,Death Note,DEATH NOTE,Light Yagami is a genius high school student w...,37.0,"['Mystery', 'Psychological', 'Supernatural', '...",854079,84.0,https://s4.anilist.co/file/anilistcdn/media/an...,"Death Note ['Mystery', 'Psychological', 'Super...",854079,8.4
3,113415,40748.0,Jujutsu Kaisen,JUJUTSU KAISEN,呪術廻戦,"A boy fights... for ""the right death.""<br>\n<b...",24.0,"['Action', 'Drama', 'Supernatural']",847065,84.0,https://s4.anilist.co/file/anilistcdn/media/an...,"JUJUTSU KAISEN ['Action', 'Drama', 'Supernatur...",847065,8.4
4,21459,31964.0,Boku no Hero Academia,My Hero Academia,僕のヒーローアカデミア,What would the world be like if 80 percent of ...,13.0,"['Action', 'Adventure', 'Comedy']",789122,76.0,https://s4.anilist.co/file/anilistcdn/media/an...,"My Hero Academia ['Action', 'Adventure', 'Come...",789122,7.6


In [27]:
matrix = df.pivot_table(index="mal_id", columns="anilist_id", values="avg_rating").fillna(0)
matrix.shape

(1499, 1499)

In [28]:
df["log_count"] = np.log1p(df["rating_count"])

df["popularity_score"] = (
    0.6 * df["avg_rating"] +
    0.3 * df["rating_count"] +
    0.1 * df["log_count"]
)


In [29]:
min_val = df["popularity_score"].min()
max_val = df["popularity_score"].max()

df["popularity_norm"] = (df["popularity_score"] - min_val) / (max_val - min_val)

In [30]:
df[["title_english", "avg_rating", "rating_count", "popularity_norm"]].head(20)


,title_english,avg_rating,rating_count,popularity_norm
0,Attack on Titan,8.5,938026,1.000000
1,Demon Slayer: Kimetsu no Yaiba,8.2,890351,0.946657
2,Death Note,8.4,854079,0.906073
3,JUJUTSU KAISEN,8.4,847065,0.898225
4,My Hero Academia,7.6,789122,0.833392
5,Hunter x Hunter (2011),8.9,753832,0.793910
6,One-Punch Man,8.3,698884,0.732428
7,Tokyo Ghoul,7.6,667290,0.697077
8,Attack on Titan Season 2,8.5,660124,0.689061
9,ONE PIECE,8.8,653791,0.681976


In [31]:
os.makedirs("model", exist_ok=True)
np.save("model/cf_features.npy", df["popularity_norm"].values)

print("Popularity-based CF features saved.")

Popularity-based CF features saved.
